In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import tropycal.tracks as tracks


In [2]:
variables = ['VMAX','SHRD','SHTD','VMPI','RSST','RHLO','IR00','DTL','HSTA']

desired_basin = 'north_atlantic' # east_pacific or north_atlantic
year_start = 2000
year_end = 2022
dt1 = datetime(year=year_start,month=1,day=1)
dt2 = datetime(year=year_end,month=12,day=31)

In [3]:
# Generate a list of storm names and ATCF IDs

basin = tracks.TrackDataset(basin=desired_basin,source='hurdat',include_btk=False)


szn_list = []
for szn in range(year_start,year_end+1):
    one_season = basin.get_season(szn).to_dataframe().set_index(['id'])
    szn_list.append(one_season)
atcf_id_sname_list = pd.concat(szn_list,axis=0)['name']

--> Starting to read in HURDAT2 data
--> Completed reading in HURDAT2 data (2.22 seconds)


In [4]:
# Grab with regular SHIPS

# Variables from SHIPS. HSTA and HEND are variables I made up for hours after RI start and hours before RI end.
variables = ['VMAX','SHRD','SHTD','VMPI','RSST','RHLO','IR00','DTL']

basin = 'north_atlantic' # east_pacific or north_atlantic

# # Read in RI cases
# RI = pd.read_csv('/Users/acheung/data/RI_Cases/RI_Events_'+basin+'.csv')

# # Convert columns to datetime format
# RI["RI Start"] = pd.to_datetime(RI["RI Start"])
# RI["RI End"] = pd.to_datetime(RI["RI End"])

list_pds = [] # Saves all the information as one big list!

for var_ind in range(len(variables)): # Loops through all variables

    SHIPS_data = pd.read_csv('/Users/acheung/data/SHIPS/SHIPS_'+variables[var_ind]+'_'+desired_basin+'.csv')
    # Generate storm names list from ATCF IDs
    SHIPS_data['Time'] = pd.to_datetime(SHIPS_data['Time'])
#     SHIPS_data = SHIPS_data.set_index('Time')
    SHIPS_data = SHIPS_data.set_index(['Time','Storm_ID'])
    SHIPS_trimmed = SHIPS_data.iloc[(SHIPS_data.index.get_level_values(0) >= dt1 ) &
                                    (SHIPS_data.index.get_level_values(0) <= dt2)]
    list_pds.append(SHIPS_trimmed)

# Concatenate all the pandas arrays
SHIPS_concat = pd.concat(list_pds,axis=1)

# # Save to CSV
# SHIPS_concat.to_csv("/Users/acheung/data/RI_Cases/RI_Events_with_SHIPS_Data_"+basin+".csv")


In [5]:
# Can open any file for generating list of storm names from SHIPS ATCF IDs
SHIPS_data = pd.read_csv('/Users/acheung/data/SHIPS/SHIPS_'+variables[0]+'_'+desired_basin+'.csv')

# Generate storm names list from ATCF IDs
names_list = []
for id_ind in range(len(SHIPS_concat.index.get_level_values(1))):
    id_now = SHIPS_concat.index.get_level_values(1)[id_ind]
    name_now = atcf_id_sname_list.loc[id_now]
    names_list.append(name_now)

In [6]:
names_pd = pd.DataFrame(names_list,index=SHIPS_concat.index,columns=['Name'])
SHIPS_with_name = pd.concat((SHIPS_concat,names_pd),axis=1)
SHIPS_with_name = SHIPS_with_name.reset_index().set_index(['Time','Storm_ID','Name'])
# # Save to CSV
SHIPS_with_name.to_csv("/Users/acheung/data/SHIPS/all_SHIPs_data_combined_"+desired_basin+".csv")

In [7]:
SHIPS_with_name

,,,VMAX,SHRD,SHTD,VMPI,RSST,RHLO,IR00,DTL
Time,Storm_ID,Name,,,,,,,,
2000-06-07 18:00:00,AL012000,UNNAMED,25,20.8,96,132,28.3,79,999,266
2000-06-08 00:00:00,AL012000,UNNAMED,25,22.3,93,131,28.2,82,22,245
2000-06-08 06:00:00,AL012000,UNNAMED,25,24.8,98,137,28.3,82,999,235
2000-06-08 12:00:00,AL012000,UNNAMED,25,24.6,111,145,28.4,81,999,257
2000-06-23 00:00:00,AL022000,UNNAMED,25,28.5,267,149,28.4,74,999,466
...,...,...,...,...,...,...,...,...,...,...
2022-11-10 12:00:00,AL172022,NICOLE,55,26.6,61,92,26.3,70,10,-78
2022-11-10 18:00:00,AL172022,NICOLE,40,25.2,42,76,25.3,72,30,5
2022-11-11 00:00:00,AL172022,NICOLE,35,25.4,46,61,24.2,69,30,-7


In [8]:
# # Grab with regular EC-SHIPS


# basin_EC = 'north_atlantic'

# RI_EC_pre = pd.read_csv('/Users/acheung/data/RI_Cases/RI_Events_'+basin_EC+'.csv')
# if basin_EC == 'north_atlantic':
#     EC_SHIPS_shear = pd.read_csv('/Users/acheung/data/EC_SHIPS/EC-SHIPS_deep_layer_shear_atl.csv')
# elif basin_EC == 'east_pacific':
#     EC_SHIPS_shear = pd.read_csv('/Users/acheung/data/EC_SHIPS/EC-SHIPS_deep_layer_shear_epac.csv')

# RI_EC = RI_EC_pre.where(RI_EC_pre['Season'] >= 2016).dropna()
    
# all_EC_shears = []
# for i in range(len(RI_EC)):
#     current_storm_shear_EC = EC_SHIPS_shear.where(EC_SHIPS_shear['Storm_ID'] ==
#                                             RI_EC.iloc[[i]]['Storm_ID'].values[0]).dropna()
#     if len(current_storm_shear_EC.where(
#         current_storm_shear_EC['Time'] == RI_EC.iloc[[i]]['RI Start'].values[0]).dropna()['Deep_Layer_Shear']) == 0:
#         # Some data is missing from EC-SHIPS, so we record NaN in these cases
#         all_EC_shears.append(np.nan)
#     else: # When data from EC-SHIPS is not missing
#         current_shear_RI = current_storm_shear_EC.where(
#             current_storm_shear_EC['Time'] == RI_EC.iloc[[i]]['RI Start'].values[0]).dropna()['Deep_Layer_Shear'].values[0]
#         all_EC_shears.append(current_shear_RI)
    
# RI_EC.insert(6, "Deep-Layer Shear (kt)", all_EC_shears)
# RI_EC.to_csv("/Users/acheung/data/RI_Cases/RI_Events_with_EC_SHIPS_shear_"+basin_EC+".csv",index=False)
